In [ ]:
import sys
sys.path.append('ift_6758_milestone_3/ift6758')
from ift6758.client.serving_client import ServingClient
from ift6758.client.game_client import Game

import requests
import pickle
import json
import pandas as pd
import numpy as np
import os

In [ ]:
Serving_client = ServingClient()

workspace = "jhd"
model_type = 'xgboost'
version = '1.1.0'

In [ ]:
## Test Download Registry Model ##
loading_response_1 = Serving_client.download_registry_model(workspace, model_type, version)
assert(loading_response_1['Loading'] == 'Success')


loading_response_2 = Serving_client.download_registry_model(workspace, model_type, '2.0.0')
assert(loading_response_2['Loading'] == 'Failure')

In [ ]:
## Test predict ##
features_dict = {
                 '1.1.0': ['goal_distance'],
                 '1.2.0': ['angle'],
                 '1.3.0': ['goal_distance', 'angle']
                }

# Any game
X_raw = pd.read_csv('ift6758/ift6758/data/wpg_v_wsh_2017021065.csv')

# Dummy to test that we keep last loaded model if model does not exist
for model_type in ['logisticregression', 'xgboost', 'dummy']:
    for version in ['1.1.0', '1.2.0', '1.3.0']:
        print(version)
        if (model_type != 'dummy') or (version == '1.3.0'):
            loading_response = Serving_client.download_registry_model(workspace, model_type, version)
            if model_type != 'dummy':
                output_path = f'./serving/models/{workspace}/{model_type}/{version}'
            else:
                output_path = f'./serving/models/jhd/xgboost/1.3.0'
            with open(os.path.join(output_path, 'model-data', 'comet-sklearn-model.pkl'), 'rb') as f:
                model = pickle.load(f)

            X = X_raw[X_raw['event_name'].isin(['Shot', 'Goal'])].reset_index(drop=True)
            X = X[features_dict[version]]

            X_dict = json.loads(X.to_json())
            data = pd.DataFrame.from_dict(X_dict)
            true_predictions = pd.DataFrame(model.predict_proba(data), columns = ['predictions_0', 'predictions_1'])
            true_predictions = true_predictions.astype('float64')

            predictions = Serving_client.predict(X)

            assert(true_predictions.equals(predictions))